### Recalling narratives with retrieval augmented generation

How do hippocampus and neocortex work together to recall narratives (and other sequences), whilst minimising the amount of detail stored in the hippocampus?

* Neocortex creates gist
* Gist plus unpredictable details stored in HPC
* Stimulus triggers recall
* Relevant event(s) retrieved from HPC
* NC elaborates details

In [ ]:
!pip install openai --upgrade
!pip install llama-index

In [ ]:
!export OPENAI_API_KEY=# add your key here

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=# add your key here,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo"
)

chat_completion.choices[0].message.content

In [ ]:
def get_output(input_text):
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model="gpt-3.5-turbo"
    )
    output_text = completion.choices[0].message.content
    return output_text

get_output("Hello world")

In [ ]:
import pandas as pd
import random

def get_stories():
    df = pd.read_csv('stories_train.csv')
    df['combined'] = df[[f'sentence{i}' for i in range(1,6)]].astype(str).agg(' '.join, axis=1)
    return df['combined'].tolist()

stories = get_stories()

In [ ]:
stories[1]

In [ ]:
get_output(f"{stories[1]} Brief summary:")

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.schema import Document

gists = [get_output(f"{story} One line summary:") for story in stories[0:10]]
docs = [Document(text=txt) for txt in gists]
index = VectorStoreIndex.from_documents(docs)

In [ ]:
for i in range(0,10):
    print("Event:")
    print(stories[i])
    print("Neocortex-generated gist:")
    print(gists[i])
    print("-----------")

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

In [ ]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index import get_response_synthesizer
from llama_index.prompts import PromptTemplate

# define custom retriever
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=1)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# vector query engine
query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information (and extrapolating from it), answer the query in detail.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

Example recall 1:

In [ ]:
response = query_engine.query("What happened when Melody's parents surprised her (in detail)?")

In [ ]:
print("Original event:")
print(stories[5])
print("\nGist retrieved from hippocampus:")
print(response.source_nodes[0].text)
print("\nNeocortex-elaborated answer:")
print(response.response)

Example recall 2:

In [ ]:
response = query_engine.query("What happened to John the pastor (in detail)?")

In [ ]:
print("Original event:")
print(stories[4])
print("\nGist retrieved from hippocampus:")
print(response.source_nodes[0].text)
print("\nNeocortex-elaborated answer:")
print(response.response)